<a href="https://colab.research.google.com/github/Natassiam/pln/blob/main/Estati%CC%81stica_e_Sema%CC%82ntica_Distribucional_Natassia_Medeiros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Aluna: Natássia Rafaelle Medeiros Siqueira matrícula: 24200953

## Exercício Estatística e Semântica Distribucional

### Importação das bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

### Carregamento do dataset

In [ ]:
df = pd.read_csv('Hotel_Reviews.csv')
reviews = df['reviews.text'].dropna().astype(str).tolist()

In [ ]:
df

,address,categories,city,country,latitude,longitude,name,postalCode,province,reviews.date,reviews.dateAdded,reviews.doRecommend,reviews.id,reviews.rating,reviews.text,reviews.title,reviews.userCity,reviews.username,reviews.userProvince
0,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-09-22T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,4.0,Pleasant 10 min walk along the sea front to th...,Good location away from the crouds,NaN,Russ (kent),NaN
1,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-04-03T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Really lovely hotel. Stayed on the very top fl...,Great hotel with Jacuzzi bath!,NaN,A Traveler,NaN
2,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2014-05-13T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Ett mycket bra hotell. Det som drog ner betyge...,Lugnt l��ge,NaN,Maud,NaN
3,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-10-27T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We stayed here for four nights in October. The...,Good location on the Lido.,NaN,Julie,NaN
4,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-03-05T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We stayed here for four nights in October. The...,������ ���������������,NaN,sungchul,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35907,1088 Powdermill Rd,"Cabins,Resorts & Vacation Cottages,Wedding Cha...",Gatlinburg,US,35.754868,-83.457295,Elk Springs Resort,37738,TN,2012-03-20T00:00:00Z,2016-06-23T16:50:58Z,NaN,NaN,5.0,Best of the Best!!!! My family and I stayed in...,Amazing time (both times!!),Charlotte,Chelsa K.,NC
35908,1088 Powdermill Rd,"Cabins,Resorts & Vacation Cottages,Wedding Cha...",Gatlinburg,US,35.754868,-83.457295,Elk Springs Resort,37738,TN,2012-03-12T00:00:00Z,2016-06-23T16:50:58Z,NaN,NaN,5.0,Amazing Vacation in Beautiful Cabin We stayed ...,Amazing time (both times!!),Charlotte,Heather D.,NC
35909,1088 Powdermill Rd,"Cabins,Resorts & Vacation Cottages,Wedding Cha...",Gatlinburg,US,35.754868,-83.457295,Elk Springs Resort,37738,TN,2015-11-03T00:00:00Z,2016-06-23T16:50:58Z,NaN,NaN,5.0,My husband and I stayed at Tree Tops which is ...,Amazing time (both times!!),Charlotte,Carol W.,NC
35910,3747 29th St S E,"Hotels-Apartment,Corporate Lodging,Hotels,Lodging",Grand Rapids,US,42.909480,-85.573590,Extended Stay America Grand Rapids - Kentwood,49512,MI,NaN,2015-12-01T19:40:54Z,NaN,NaN,0.0,to share your opinion of this businesswith YP ...,NaN,NaN,write a review,NaN


### Definição das matrizes, das 5 palavras e a medida do Cosseno entre:
- location e price
- cleaning e service

In [ ]:
# Ensure NLTK data is properly downloaded
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt', quiet=True)

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords', quiet=True)

# Load data
try:
    df = pd.read_csv('Hotel_Reviews.csv')
    reviews = df['reviews.text'].dropna().astype(str).tolist()
except Exception as e:
    print(f"Error loading data: {e}")
    reviews = []

# Preprocessing
stop_words = set(stopwords.words('english'))

def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    try:
        tokens = word_tokenize(text)
    except:
        tokens = text.split()  # fallback if tokenization fails
    tokens = [word for word in tokens if word not in stop_words and len(word) > 2]
    return tokens

# Target words
target_words = ['location', 'price', 'cleaning', 'service']

# Term-context matrix
window_size = 5
term_context_matrix = defaultdict(lambda: defaultdict(int))
vocab = set()

for review in reviews:
    tokens = preprocess(review)
    for i, word in enumerate(tokens):
        if word in target_words:
            start = max(0, i - window_size)
            end = min(len(tokens), i + window_size + 1)
            context = tokens[start:i] + tokens[i+1:end]

            for context_word in context:
                term_context_matrix[word][context_word] += 1
                vocab.add(context_word)

# Create matrix
vocab = sorted(vocab)
word_to_idx = {word: i for i, word in enumerate(vocab)}
target_to_idx = {word: i for i, word in enumerate(target_words)}

matrix = np.zeros((len(target_words), len(vocab)))

for target_word, context_counts in term_context_matrix.items():
    for context_word, count in context_counts.items():
        if context_word in word_to_idx:
            matrix[target_to_idx[target_word], word_to_idx[context_word]] = count

# Cosine similarity function
def cosine_sim(word1, word2):
    if word1 not in target_to_idx or word2 not in target_to_idx:
        return 0.0  # if words not found

    vec1 = matrix[target_to_idx[word1]]
    vec2 = matrix[target_to_idx[word2]]

    # Handle zero vectors
    if np.all(vec1 == 0) or np.all(vec2 == 0):
        return 0.0

    return cosine_similarity([vec1], [vec2])[0][0]

# Calculate similarities
similarities = {
    'location_price': cosine_sim('location', 'price'),
    'cleaning_service': cosine_sim('cleaning', 'service')
}

# Results
print("Term-Context Matrix Shape:", matrix.shape)
print("\nCosine Similarities:")
for pair, sim in similarities.items():
    print(f"{pair}: {sim:.4f}")

# Save results
np.save('term_context_matrix.npy', matrix)
with open('vocabulary.txt', 'w') as f:
    f.write('\n'.join(vocab))

## Exercício Representação de Textos

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import json
import re


# Carregar os dados
df = pd.read_csv('Hotel_Reviews.csv')

In [4]:
# Função para pré-processamento de texto
def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    # Converter para minúsculas
    text = text.lower()
    # Remover caracteres especiais e números
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remover espaços extras
    text = re.sub(r'\s+', ' ', text).strip()
    return text


# Selecionar apenas as colunas relevantes e remover linhas com comentários vazios
reviews = df[['reviews.text', 'reviews.rating']].dropna()
reviews['cleaned_text'] = reviews['reviews.text'].apply(preprocess_text)


In [5]:
# 1. Representação TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(reviews['cleaned_text'])
tfidf_features = tfidf_vectorizer.get_feature_names_out()

# Converter para formato serializável
tfidf_data = {
    "features": tfidf_features.tolist(),
    "vectors": tfidf_matrix.toarray().tolist(),
    "reviews": reviews['reviews.text'].tolist(),
    "ratings": reviews['reviews.rating'].tolist()
}

# Salvar em JSON
with open('hotel_reviews_tfidf.json', 'w') as f:
    json.dump(tfidf_data, f, indent=2)


In [6]:
# 2. Representação Bag of Words (BoW)
bow_vectorizer = CountVectorizer(max_features=1000, stop_words='english')
bow_matrix = bow_vectorizer.fit_transform(reviews['cleaned_text'])
bow_features = bow_vectorizer.get_feature_names_out()

bow_data = {
    "features": bow_features.tolist(),
    "vectors": bow_matrix.toarray().tolist(),
    "reviews": reviews['reviews.text'].tolist(),
    "ratings": reviews['reviews.rating'].tolist()
}

with open('hotel_reviews_bow.json', 'w') as f:
    json.dump(bow_data, f, indent=2)


In [7]:
# 3. Representação Bag of N-Grams (Bigramas e Trigramas)
ngram_vectorizer = CountVectorizer(ngram_range=(2, 3), max_features=1000, stop_words='english')
ngram_matrix = ngram_vectorizer.fit_transform(reviews['cleaned_text'])
ngram_features = ngram_vectorizer.get_feature_names_out()

ngram_data = {
    "features": ngram_features.tolist(),
    "vectors": ngram_matrix.toarray().tolist(),
    "reviews": reviews['reviews.text'].tolist(),
    "ratings": reviews['reviews.rating'].tolist()
}

with open('hotel_reviews_ngram.json', 'w') as f:
    json.dump(ngram_data, f, indent=2)


In [8]:
print("Processamento concluído. Arquivos JSON gerados:")
print("- hotel_reviews_tfidf.json")
print("- hotel_reviews_bow.json")
print("- hotel_reviews_ngram.json")

Processamento concluído. Arquivos JSON gerados:
- hotel_reviews_tfidf.json
- hotel_reviews_bow.json
- hotel_reviews_ngram.json
